In [ ]:
# !pip install torch torchvisioaudio --index-url https://download.pytorch.org/whl/cu126
# !pip install sentence-transformers

In [20]:
!pip show sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: sentence-transformers
Version: 2.6.1
Summary: Multilingual text embeddings
Home-page: https://www.SBERT.net
Author: Nils Reimers
Author-email: info@nils-reimers.de
License: Apache License 2.0
Location: /home/jie/anaconda3/envs/agent/lib/python3.11/site-packages
Requires: huggingface-hub, numpy, Pillow, scikit-learn, scipy, torch, tqdm, transformers
Required-by: langchain-huggingface, llama-index-embeddings-huggingface


In [ ]:
# !pip install transformers modelscope langchain sentence_transformers langchain-huggingface 

# ## 根据langchain的文档，安装chroma
# ## https://python.langchain.com/docs/integrations/vectorstores/chroma/
# !pip install -qU "langchain-chroma>=0.1.2"

In [1]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders.text import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

# embedding 模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('BAAI/bge-m3')

/home/jie/anaconda3/envs/agent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load the document and split it into chunks
chunks = []
PREFIX = "以下是{file_name}年政府工作报告的节选部分:"
fold = "data"
for file in os.listdir(fold):
    file_name = os.path.join(fold, file)
    if file.endswith(".txt"):
        prefix = PREFIX.format(file_name=file.split(".")[0])
    else:
        continue
    loader = TextLoader(file_name)
    documents = loader.load()
    
    # split it into chunks
    text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    tmp_chunks = (text_splitter.split_documents(documents))
    for chunk in tmp_chunks:
        chunk.page_content = prefix + chunk.page_content
    
    chunks.extend(tmp_chunks)

Created a chunk of size 593, which is longer than the specified 200
Created a chunk of size 903, which is longer than the specified 200
Created a chunk of size 396, which is longer than the specified 200
Created a chunk of size 584, which is longer than the specified 200
Created a chunk of size 398, which is longer than the specified 200
Created a chunk of size 763, which is longer than the specified 200
Created a chunk of size 791, which is longer than the specified 200
Created a chunk of size 857, which is longer than the specified 200
Created a chunk of size 1169, which is longer than the specified 200
Created a chunk of size 380, which is longer than the specified 200
Created a chunk of size 290, which is longer than the specified 200
Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 1892, which is longer than the specified 200
Created a chunk of size 662, which is longer than the specified 200
Created a chunk of size 1295, which is longer 

In [4]:
prefix

'以下是内蒙古自治区2024年政府工作报告的节选部分:'

In [5]:
len(chunks)

442

In [6]:
chunks[0], chunks[-1]

(Document(metadata={'source': 'data/七台河市2024.txt'}, page_content='以下是七台河市2024年政府工作报告的节选部分:七台河市政府工作报告（2024年1月12日 李兵）\n2024年04月01日 17:21   来源：七台河市政府网   \n[手机看新闻][字号 大 中 小][打印本稿]\n\u3000\u3000——2024年1月12日在七台河市第十二届人民代表大会第三次会议上\n\n\u3000\u3000李兵\n\n\u3000\u3000各位代表:\n\n\u3000\u3000现在,我代表市政府向大会报告工作,请予审议,并请政协委员提出意见。\n\n\u3000\u3000一、2023年工作回顾'),
 Document(metadata={'source': 'data/内蒙古自治区2024.txt'}, page_content='以下是内蒙古自治区2024年政府工作报告的节选部分:“七个作模范”：在感党恩听党话、紧跟习近平总书记奋进新征程上作模范，在铸牢中华民族共同体意识上作模范，在民族地区推进中国式现代化建设中作模范，在边疆民族地区走向共同富裕的道路上作模范，在兴边稳边固边上作模范，在边疆地区联通国内国际双循环上作模范，在弘扬新风正气上作模范。'))

In [7]:
print(chunks[0].page_content)

以下是七台河市2024年政府工作报告的节选部分:七台河市政府工作报告（2024年1月12日 李兵）
2024年04月01日 17:21   来源：七台河市政府网   
[手机看新闻][字号 大 中 小][打印本稿]
　　——2024年1月12日在七台河市第十二届人民代表大会第三次会议上

　　李兵

　　各位代表:

　　现在,我代表市政府向大会报告工作,请予审议,并请政协委员提出意见。

　　一、2023年工作回顾


In [8]:
print(chunks[-1].page_content)

以下是内蒙古自治区2024年政府工作报告的节选部分:“七个作模范”：在感党恩听党话、紧跟习近平总书记奋进新征程上作模范，在铸牢中华民族共同体意识上作模范，在民族地区推进中国式现代化建设中作模范，在边疆民族地区走向共同富裕的道路上作模范，在兴边稳边固边上作模范，在边疆地区联通国内国际双循环上作模范，在弘扬新风正气上作模范。


In [9]:
model_dir

'/home/jie/.cache/modelscope/hub/models/BAAI/bge-m3'

In [10]:
# create the open-source embedding function
bge_m3_embedding = HuggingFaceEmbeddings(
    model_name=model_dir, model_kwargs={
        "trust_remote_code": True, 
        }
)

In [11]:
bge_m3_embedding.embed_query("Hello")

[-0.03157564997673035,
 0.024295102804899216,
 -0.028296565636992455,
 0.0174085795879364,
 -0.021942604333162308,
 -0.03826744854450226,
 -0.036494407802820206,
 -0.027303434908390045,
 0.003755931742489338,
 0.0074757058173418045,
 0.008294942788779736,
 0.015207814052700996,
 0.02142738737165928,
 -0.00996940117329359,
 0.005842524114996195,
 -0.03261396661400795,
 0.0274842269718647,
 -0.024674389511346817,
 -0.014383852481842041,
 -0.040320757776498795,
 -0.05038909986615181,
 -0.013991760089993477,
 0.022246234118938446,
 -0.013500478118658066,
 0.02265114150941372,
 0.054930299520492554,
 -0.042421456426382065,
 0.006691767368465662,
 0.0037921173498034477,
 -0.008699668571352959,
 0.04470004141330719,
 0.06099865958094597,
 -0.0042753140442073345,
 -0.043184686452150345,
 -0.01622086949646473,
 -0.04193943366408348,
 0.0017361853970214725,
 -0.020693695172667503,
 -0.05081554874777794,
 0.030132632702589035,
 0.013580990023911,
 0.012070690281689167,
 0.01450493186712265,
 -0.0

In [12]:
len(bge_m3_embedding.embed_query("Hello"))

1024

In [13]:
import torch

torch.cuda.is_available()

True

In [ ]:
# # https://python.langchain.com/docs/how_to/vectorstores/
# vector_store = Chroma(
#     collection_name="demo_db",
#     embedding_function=bge_m3_embedding,
#     # Where to save data locally, remove if not necessary
#     persist_directory="./chroma_langchain_db",  
# )

# # 默认使用GPU计算，除非torch不能使用cuda
# vector_store.add_documents(chunks)

['b82e3b48-9e22-4369-a56e-c657a7209aee',
 'd60b75a2-dbbe-4ceb-83dd-86a2eb31b4af',
 '70f7066c-7c96-4db0-abbe-994e4177d983',
 '433624d1-1879-4c54-a68f-7e6f880b79e5',
 '9cab692e-cb6a-4a27-b429-46b8cd1dcbca',
 '27eb8d1f-9c47-4758-ae56-72c7e5102594',
 'bffee37a-a885-4b68-a10a-e53c7980c69e',
 'e9d657d0-9a05-4e33-849f-ca18fd15aa79',
 '92b019fa-0f2d-4ba9-be12-a407dd2c959c',
 'cd283bea-6a21-41ee-b758-63f98b7bb752',
 '8e102c49-a913-42aa-b5ea-ed615f2c2d51',
 '3d88f655-0903-44f9-8816-6c3c68ad7449',
 'e1ece296-0d76-4841-a812-7ff6bd997020',
 'd371ee17-70b2-44ed-9f08-4329a9c941df',
 '575c0ff4-5578-47a9-806f-2f43247948e3',
 'a7752bd7-ee2a-4fb5-b24f-24d6b43a54f7',
 '0f447ca6-cf07-4970-9822-e5965504abf2',
 '981d3fef-ce0b-4854-b761-4a08f8a7c64e',
 'dd0c3151-36ad-4895-ba15-0a552d7313e1',
 '36bfe576-ec58-42c6-b8c2-3147182cd959',
 'ad386373-2ee8-4ae0-a930-79bdfc4521c8',
 '55d663b1-4229-4c8d-bcb3-69ce66a77bd8',
 '7fba571b-2251-477c-9c66-19adda759c31',
 '575ef89c-0f89-420d-b284-9d41e72ef622',
 '56b1113d-b30a-

In [15]:
# 从本地加载
vector_store = Chroma(
    collection_name="demo_db",          # 必须和保存时一致
    embedding_function=bge_m3_embedding,   # 和保存时使用的模型一致
    persist_directory="chroma_langchain_db"          # 保存路径
)

In [16]:
vector_store.similarity_search("生产总值")

[Document(id='b0cb28f9-dce7-4c6a-a3da-d4724f10afed', metadata={'source': 'data/上海市2023.txt'}, page_content='以下是上海市2023年政府工作报告的节选部分:——城市综合实力跃上新台阶。全市生产总值突破4万亿元大关，2022年预计达到4.45万亿元，人均生产总值达到17.8万元左右。地方一般公共预算收入达到7608.2亿元。金融市场交易总额从2017年的1428.4万亿元增加到近3000万亿元。口岸贸易总额突破10万亿元、占全球比重提高到3.6%左右，保持世界城市首位。上海港集装箱吞吐量达到4730.3万标准箱、连续13年排名世界第一。全社会研发经费支出相当于全市生产总值的比例提高到4.2%左右，每万人口高价值发明专利拥有量达到40件左右。'),
 Document(id='4b94f95a-b9f4-46e1-adfa-99525d786f4e', metadata={'source': 'data/上海市2022.txt'}, page_content='以下是上海市2022年政府工作报告的节选部分:一年来，全市经济社会平稳健康发展，呈现稳中加固、稳中有进、稳中向好态势。经济持续回升向好。全市生产总值突破4万亿元大关，达到4.32万亿元，增长8.1%。地方一般公共预算收入增长10.3%，居民消费价格上涨1.2%，城镇调查失业率明显低于年初预期目标，新增就业岗位63.5万个。发展新动能持续成长。全社会研发经费支出相当于全市生产总值的比例保持在4.1%左右，集成电路、生物医药、人工智能三大产业制造业产值增长18.3%，新增市场主体快速增长，每千人拥有的企业数位居全国第一。人民生活水平持续提高。居民人均可支配收入增长8%。生态环境持续改善。单位生产总值能耗进一步下降，PM2.5年均浓度为27微克/立方米、下降15.6%。'),
 Document(id='fb3334bc-e325-4d25-823a-c91c9c8cf52c', metadata={'source': 'data/上海市2024.txt'}, page_content='以下是上海市2024年政府工作报告的节选部分:一年来，全市经济社会发展稳中有进、稳中向好。一是经济运行稳步恢复。全市生产总值达到

In [17]:
query = "上海市2024年工作报告中提到的国民生成总值是多少？"
relevant_docs = vector_store.similarity_search(query)
relevant_docs

[Document(id='383afbf5-4685-4a88-991e-1509820e8316', metadata={'source': 'data/上海市2024.txt'}, page_content='以下是上海市2024年政府工作报告的节选部分:综合各方面因素，建议今年全市经济社会发展的主要预期目标是：全市生产总值增长5%左右，地方一般公共预算收入增长5%，全社会研发经费支出相当于全市生产总值的比例达到4.5%左右，城镇调查失业率5%以内，居民人均可支配收入增长与经济增长保持同步，居民消费价格涨幅3%左右，单位生产总值能耗、单位生产总值二氧化碳排放量进一步下降，主要污染物重点工程减排量完成国家下达目标。\n\n今年重点做好以下工作。'),
 Document(id='fb3334bc-e325-4d25-823a-c91c9c8cf52c', metadata={'source': 'data/上海市2024.txt'}, page_content='以下是上海市2024年政府工作报告的节选部分:一年来，全市经济社会发展稳中有进、稳中向好。一是经济运行稳步恢复。全市生产总值达到4.72万亿元，增长5%。地方一般公共预算收入增长9.3%，居民消费价格上涨0.3%，城镇调查失业率平均为4.5%。二是发展新动能稳步壮大。工业战略性新兴产业总产值占规模以上工业总产值比重达到43.9%，集成电路、生物医药、人工智能三大先导产业规模达到1.6万亿元。全社会研发经费支出相当于全市生产总值的比例达到4.4%左右，每万人口高价值发明专利拥有量提高到50.2件。三是改革开放红利稳步释放。外贸进出口总额达到4.2万亿元、增长0.7%，实际使用外资240.9亿美元、创历史新高。金融市场交易总额达到3373.6万亿元、增长15%。跨国公司地区总部、外资研发中心分别新增65家和30家，累计分别达到956家和561家。首批创新型企业总部达到40家。四是民生福祉稳步改善。居民人均可支配收入达到8.5万元、增长6.6%，增速快于经济增长，环境空气质量优良率达到87.7%、提高0.6个百分点，新增公园162座、累计达到832座。'),
 Document(id='670cfd55-7b2d-4bb6-a6f0-320f72405fa6', metadata={'source': 'da

In [18]:
vector_store.similarity_search("上海市2024年工作报告中提到的GDP是多少？")

[Document(id='383afbf5-4685-4a88-991e-1509820e8316', metadata={'source': 'data/上海市2024.txt'}, page_content='以下是上海市2024年政府工作报告的节选部分:综合各方面因素，建议今年全市经济社会发展的主要预期目标是：全市生产总值增长5%左右，地方一般公共预算收入增长5%，全社会研发经费支出相当于全市生产总值的比例达到4.5%左右，城镇调查失业率5%以内，居民人均可支配收入增长与经济增长保持同步，居民消费价格涨幅3%左右，单位生产总值能耗、单位生产总值二氧化碳排放量进一步下降，主要污染物重点工程减排量完成国家下达目标。\n\n今年重点做好以下工作。'),
 Document(id='fb3334bc-e325-4d25-823a-c91c9c8cf52c', metadata={'source': 'data/上海市2024.txt'}, page_content='以下是上海市2024年政府工作报告的节选部分:一年来，全市经济社会发展稳中有进、稳中向好。一是经济运行稳步恢复。全市生产总值达到4.72万亿元，增长5%。地方一般公共预算收入增长9.3%，居民消费价格上涨0.3%，城镇调查失业率平均为4.5%。二是发展新动能稳步壮大。工业战略性新兴产业总产值占规模以上工业总产值比重达到43.9%，集成电路、生物医药、人工智能三大先导产业规模达到1.6万亿元。全社会研发经费支出相当于全市生产总值的比例达到4.4%左右，每万人口高价值发明专利拥有量提高到50.2件。三是改革开放红利稳步释放。外贸进出口总额达到4.2万亿元、增长0.7%，实际使用外资240.9亿美元、创历史新高。金融市场交易总额达到3373.6万亿元、增长15%。跨国公司地区总部、外资研发中心分别新增65家和30家，累计分别达到956家和561家。首批创新型企业总部达到40家。四是民生福祉稳步改善。居民人均可支配收入达到8.5万元、增长6.6%，增速快于经济增长，环境空气质量优良率达到87.7%、提高0.6个百分点，新增公园162座、累计达到832座。'),
 Document(id='670cfd55-7b2d-4bb6-a6f0-320f72405fa6', metadata={'source': 'da

In [19]:
query = "上海市2024年工作报告中提到新增就业岗位多少个？"
relevant_docs = vector_store.similarity_search(query)
relevant_docs

[Document(id='e135765a-68b6-4fd4-8106-9873cafb8117', metadata={'source': 'data/上海市2024.txt'}, page_content='以下是上海市2024年政府工作报告的节选部分:加强就业服务和社会保障。更加突出就业优先导向，完善创业担保贷款、职业培训补贴等稳就业政策，城镇新增就业55万人以上。精准做好高校毕业生、就业困难人员等重点群体就业帮扶，实施灵活就业人员服务保障行动。统筹调整养老金、医保、低保等民生保障待遇标准。加强低收入人口动态监测，开展分层分类社会救助工作。'),
 Document(id='8942c9ad-02a2-4f03-b575-c0f96930c474', metadata={'source': 'data/上海市2023.txt'}, page_content='以下是上海市2023年政府工作报告的节选部分:促进高质量充分就业和市民持续增收。健全就业促进机制，强化创业带动就业，新增就业岗位55万个以上。完善高校毕业生、退役军人等重点群体就业支持体系，加强就业困难群体帮扶和灵活就业、新就业形态劳动者权益保障，健全职业技能培训政策体系。推进体现效率、促进公平的收入分配体系建设。'),
 Document(id='f178c33e-8cd1-47f3-9246-ab172dd1573b', metadata={'source': 'data/上海市2024.txt'}, page_content='以下是上海市2024年政府工作报告的节选部分:基本民生保障持续加强。稳岗扩岗、创业扶持、技能培训等稳就业政策加快落实，公共就业招聘新平台上线运行，新增就业岗位60.6万个，建成社区就业服务站点227个，高校毕业生等重点群体就业支持体系持续健全。养老金、医保、低保等民生保障待遇水平稳步提高，灵活就业人员在沪参保全面覆盖，及时为困难群众发放价格临时补贴和各类帮困补助。残疾人社会保障制度和关爱服务体系进一步完善。'),
 Document(id='0b5d7cf9-7ab7-46a8-ae5b-04dfa13ff04f', metadata={'source': 'data/上海市2022.txt'}, page_content='以下是上海市2022年政府工作报告的节选部分:

## LLM

In [20]:
from langchain_community.llms import Ollama
from langchain.schema import SystemMessage, HumanMessage

llm = Ollama(model="qwen2.5:7b", temperature=0)

# prompt = [SystemMessage("You are a helpful assistant"), HumanMessage("Who are you?")]
# print(llm.invoke(prompt))

In [21]:
llm.invoke("hello, who are you?")

"Hello! I'm Qwen, an AI assistant created by Alibaba Cloud. How can I assist you today?"

### 手动实现

In [22]:
# context = "\n\n".join([doc.page_content for doc in relevant_docs])
PROMPT = """
你是一个智能助手，擅长从提供的背景知识中提取关键信息来回答问题。
请根据以下提供的背景知识回答问题，如果无法找到答案，请直接说“我不知道”。

背景知识：
{context}

问题：
{query}

答案：
"""

query = "上海市2024年工作报告中提到新增就业岗位多少个？"# 新增就业岗位？？？万个
relevant_docs = vector_store.similarity_search(query)

In [23]:
relevant_docs

[Document(id='e135765a-68b6-4fd4-8106-9873cafb8117', metadata={'source': 'data/上海市2024.txt'}, page_content='以下是上海市2024年政府工作报告的节选部分:加强就业服务和社会保障。更加突出就业优先导向，完善创业担保贷款、职业培训补贴等稳就业政策，城镇新增就业55万人以上。精准做好高校毕业生、就业困难人员等重点群体就业帮扶，实施灵活就业人员服务保障行动。统筹调整养老金、医保、低保等民生保障待遇标准。加强低收入人口动态监测，开展分层分类社会救助工作。'),
 Document(id='8942c9ad-02a2-4f03-b575-c0f96930c474', metadata={'source': 'data/上海市2023.txt'}, page_content='以下是上海市2023年政府工作报告的节选部分:促进高质量充分就业和市民持续增收。健全就业促进机制，强化创业带动就业，新增就业岗位55万个以上。完善高校毕业生、退役军人等重点群体就业支持体系，加强就业困难群体帮扶和灵活就业、新就业形态劳动者权益保障，健全职业技能培训政策体系。推进体现效率、促进公平的收入分配体系建设。'),
 Document(id='f178c33e-8cd1-47f3-9246-ab172dd1573b', metadata={'source': 'data/上海市2024.txt'}, page_content='以下是上海市2024年政府工作报告的节选部分:基本民生保障持续加强。稳岗扩岗、创业扶持、技能培训等稳就业政策加快落实，公共就业招聘新平台上线运行，新增就业岗位60.6万个，建成社区就业服务站点227个，高校毕业生等重点群体就业支持体系持续健全。养老金、医保、低保等民生保障待遇水平稳步提高，灵活就业人员在沪参保全面覆盖，及时为困难群众发放价格临时补贴和各类帮困补助。残疾人社会保障制度和关爱服务体系进一步完善。'),
 Document(id='0b5d7cf9-7ab7-46a8-ae5b-04dfa13ff04f', metadata={'source': 'data/上海市2022.txt'}, page_content='以下是上海市2022年政府工作报告的节选部分:

In [24]:
context = "\n\n".join([doc.page_content for doc in relevant_docs])
print(context)

以下是上海市2024年政府工作报告的节选部分:加强就业服务和社会保障。更加突出就业优先导向，完善创业担保贷款、职业培训补贴等稳就业政策，城镇新增就业55万人以上。精准做好高校毕业生、就业困难人员等重点群体就业帮扶，实施灵活就业人员服务保障行动。统筹调整养老金、医保、低保等民生保障待遇标准。加强低收入人口动态监测，开展分层分类社会救助工作。

以下是上海市2023年政府工作报告的节选部分:促进高质量充分就业和市民持续增收。健全就业促进机制，强化创业带动就业，新增就业岗位55万个以上。完善高校毕业生、退役军人等重点群体就业支持体系，加强就业困难群体帮扶和灵活就业、新就业形态劳动者权益保障，健全职业技能培训政策体系。推进体现效率、促进公平的收入分配体系建设。

以下是上海市2024年政府工作报告的节选部分:基本民生保障持续加强。稳岗扩岗、创业扶持、技能培训等稳就业政策加快落实，公共就业招聘新平台上线运行，新增就业岗位60.6万个，建成社区就业服务站点227个，高校毕业生等重点群体就业支持体系持续健全。养老金、医保、低保等民生保障待遇水平稳步提高，灵活就业人员在沪参保全面覆盖，及时为困难群众发放价格临时补贴和各类帮困补助。残疾人社会保障制度和关爱服务体系进一步完善。

以下是上海市2022年政府工作报告的节选部分:提升就业服务和社会保障水平。加大就业优先政策实施力度，落细落实减负稳岗扩就业等措施，新增就业岗位55万个以上。开展就业专项帮扶、职业技能提升等行动，促进高校毕业生、退役军人等重点群体就业创业，做好失业人员再就业工作，维护新就业形态劳动者权益。全力做好第四十六届世界技能大赛筹办工作，努力办成一届富有新意、影响广泛的世界技能大赛。按照国家部署，做好基本养老保险全国统筹实施工作。优化收入分配结构，统筹提高养老金、医保、低保等社保待遇标准，促进慈善事业发展。


In [25]:
print(PROMPT.format(context=context, query=query).lstrip())

你是一个智能助手，擅长从提供的背景知识中提取关键信息来回答问题。
请根据以下提供的背景知识回答问题，如果无法找到答案，请直接说“我不知道”。

背景知识：
以下是上海市2024年政府工作报告的节选部分:加强就业服务和社会保障。更加突出就业优先导向，完善创业担保贷款、职业培训补贴等稳就业政策，城镇新增就业55万人以上。精准做好高校毕业生、就业困难人员等重点群体就业帮扶，实施灵活就业人员服务保障行动。统筹调整养老金、医保、低保等民生保障待遇标准。加强低收入人口动态监测，开展分层分类社会救助工作。

以下是上海市2023年政府工作报告的节选部分:促进高质量充分就业和市民持续增收。健全就业促进机制，强化创业带动就业，新增就业岗位55万个以上。完善高校毕业生、退役军人等重点群体就业支持体系，加强就业困难群体帮扶和灵活就业、新就业形态劳动者权益保障，健全职业技能培训政策体系。推进体现效率、促进公平的收入分配体系建设。

以下是上海市2024年政府工作报告的节选部分:基本民生保障持续加强。稳岗扩岗、创业扶持、技能培训等稳就业政策加快落实，公共就业招聘新平台上线运行，新增就业岗位60.6万个，建成社区就业服务站点227个，高校毕业生等重点群体就业支持体系持续健全。养老金、医保、低保等民生保障待遇水平稳步提高，灵活就业人员在沪参保全面覆盖，及时为困难群众发放价格临时补贴和各类帮困补助。残疾人社会保障制度和关爱服务体系进一步完善。

以下是上海市2022年政府工作报告的节选部分:提升就业服务和社会保障水平。加大就业优先政策实施力度，落细落实减负稳岗扩就业等措施，新增就业岗位55万个以上。开展就业专项帮扶、职业技能提升等行动，促进高校毕业生、退役军人等重点群体就业创业，做好失业人员再就业工作，维护新就业形态劳动者权益。全力做好第四十六届世界技能大赛筹办工作，努力办成一届富有新意、影响广泛的世界技能大赛。按照国家部署，做好基本养老保险全国统筹实施工作。优化收入分配结构，统筹提高养老金、医保、低保等社保待遇标准，促进慈善事业发展。

问题：
上海市2024年工作报告中提到新增就业岗位多少个？

答案：



In [26]:
answer = llm.invoke(
    PROMPT.format(context=context, query=query).lstrip()
)

print(answer)

上海市2024年工作报告的节选部分提到，新增就业岗位60.6万个。


### langchain封装RAG问答模块

In [27]:
from langchain.chains import RetrievalQA


# 创建 RAG QA Chain
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())

# 直接提问
query = "上海市2024年工作报告中提到新增就业岗位多少个？"
answer = qa.invoke(query)

print(answer)


{'query': '上海市2024年工作报告中提到新增就业岗位多少个？', 'result': '上海市2024年工作报告的节选部分提到，新增就业岗位60.6万个。'}
